### Imports

In [1]:
import json
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from pyteal import compileTeal, Mode
from contract import approval_program, clear_state_program
from deployment_functions import *
from helper_functions import *

### Create Testing Account

In [2]:
def generate_algorand_keypair():
    test_private_key, test_address = account.generate_account()
    print("My address: {}".format(test_address))
    print("My private key: {}".format(test_private_key))
    print("My passphrase: {}".format(mnemonic.from_private_key(test_private_key)))    
    test_mnemonic = mnemonic.from_private_key(test_private_key)

    with open("credentials.json", "r+") as jsonFile:
        data = json.load(jsonFile)

        data["test_address"] = test_address
        data["test_mnemonic"] = test_mnemonic

        jsonFile.seek(0)  # bring cursor to the top
        json.dump(data, jsonFile, indent = 6)
        jsonFile.truncate()

### Initialize Algod Client

In [3]:
# user declared algod connection parameters. Node must have EnableDeveloperAPI set to true in its config
algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

# initialize an algodClient
algod_client = algod.AlgodClient(algod_token, algod_address)

### Credentials

In [4]:
# Opening JSON file
f = open('credentials.json')
 
# returns JSON object as a dictionary
data = json.load(f)
 
creator_address = data["creator_address"]
creator_mnemonic = data["creator_mnemonic"]
creator_private_key = mnemonic.to_private_key(creator_mnemonic)
app_id = data["app_id"]
asset_id = data["asset_id"]
application_address = data["application_address"]
test_address = data["test_address"]
test_mnemonic = data["test_mnemonic"]
test_private_key = mnemonic.to_private_key(test_mnemonic)
# Closing file
f.close()

### Create Testing asset

In [5]:
def create_asset_wrapper(private_key):
    asset_id = create_asset(algod_client, private_key)

    with open("credentials.json", "r+") as jsonFile:
        data = json.load(jsonFile)

        data['asset_id'] = asset_id

        jsonFile.seek(0)  # bring cursor to the top
        json.dump(data, jsonFile, indent = 6)
        jsonFile.truncate()

### Compiling smart contract and declaring schema

In [6]:
# get PyTeal approval program
approval_program_ast = approval_program()
# compile program to TEAL assembly
approval_program_teal = compileTeal(approval_program_ast, mode=Mode.Application, version=6)
# compile program to binary
approval_program_compiled = compile_program(algod_client, approval_program_teal)


# get PyTeal clear state program
clear_state_program_ast = clear_state_program()
# compile program to TEAL assembly
clear_state_program_teal = compileTeal(clear_state_program_ast, mode=Mode.Application, version=6)
# compile program to binary
clear_state_program_compiled = compile_program(algod_client, clear_state_program_teal)


# declare application state storage (immutable)
local_ints = 0
local_bytes = 0
global_ints = 0
global_bytes = 0
global_schema = transaction.StateSchema(global_ints, global_bytes)
local_schema = transaction.StateSchema(local_ints, local_bytes)

### Create the Smart Contract

In [7]:
def create_app_wrapper(private_key):
    # create new application
    app_id = create_app(
        algod_client,
        private_key,
        approval_program_compiled,
        clear_state_program_compiled,
        global_schema,
        local_schema,
    )

    with open("credentials.json", "r+") as jsonFile:
        data = json.load(jsonFile)

        data['app_id'] = app_id

        jsonFile.seek(0)  # bring cursor to the top
        json.dump(data, jsonFile, indent = 6)
        jsonFile.truncate()

### Update the Smart Contract

In [8]:
def update_app_wrapper(private_key):

    application_address = update_app(algod_client, 
                                    private_key, 
                                    app_id, 
                                    approval_program_compiled, 
                                    clear_state_program_compiled)


    with open("credentials.json", "r+") as jsonFile:
        data = json.load(jsonFile)

        data['application_address'] = application_address

        jsonFile.seek(0)  # bring cursor to the top
        json.dump(data, jsonFile, indent = 6)
        jsonFile.truncate()

update_app_wrapper(test_private_key)

Waiting for confirmation...
Transaction QLUB4Q2AB2IZYKZG4SNCXEMO3RSIRL26MXXRGYAOEYUI63NVFDHQ confirmed in round 20707585.
Application was modified with app_id: 81470745 and with application address: 56IHDC5FJKK6QK2KSXH2WCXNQTUTGMFIC3F5XPHQDWOI6OWMYYOUOIBIX4


### Opt user into Smart Contract

In [9]:
if not is_opted_in_app(algod_client, test_address, app_id):

    # opt into application from pool account
    result = opt_in_app(algod_client, test_private_key, app_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)


### Call the Smart Contract

In [10]:
# call application without arguments
app_args = [b"deposit", b"AssetTxn", 1]

call_app(algod_client, 
        test_private_key, 
        app_id, 
        app_args,
        rekey_to=application_address,
        foreign_assets=[asset_id]
        )


Call from account: HNDUHBASQYQTWKHGQGH7UQCYKA2Y75HKTKK2TL4DBDWO3YAKQMV2Y4BVFE
Waiting for confirmation...
Transaction OXTG2MDHXTPREBMDDA3QTOUD4FIKICPCTNJ3KKDHTPN2TIW2O7XA confirmed in round 20707587.


{'confirmed-round': 20707587,
 'inner-txns': [{'pool-error': '',
   'txn': {'txn': {'aamt': 1,
     'arcv': '56IHDC5FJKK6QK2KSXH2WCXNQTUTGMFIC3F5XPHQDWOI6OWMYYOUOIBIX4',
     'fv': 20707585,
     'lv': 20708585,
     'rekey': 'HNDUHBASQYQTWKHGQGH7UQCYKA2Y75HKTKK2TL4DBDWO3YAKQMV2Y4BVFE',
     'snd': 'HNDUHBASQYQTWKHGQGH7UQCYKA2Y75HKTKK2TL4DBDWO3YAKQMV2Y4BVFE',
     'type': 'axfer',
     'xaid': 81470512}}}],
 'pool-error': '',
 'txn': {'sig': 'nJ4rcPLyguQ0MwuE0ozQZzjQOY/0vplCzX75hpxy8Rko8P6hfqdWKtmSHISnwzz5m5aFQOIC8qiQfrxyghdTAA==',
  'txn': {'apaa': ['ZGVwb3NpdA==', 'QXNzZXRUeG4=', 'AAAAAAAAAAE='],
   'apas': [81470512],
   'apid': 81470745,
   'fee': 891000,
   'fv': 20707585,
   'gen': 'testnet-v1.0',
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'lv': 20708585,
   'rekey': '56IHDC5FJKK6QK2KSXH2WCXNQTUTGMFIC3F5XPHQDWOI6OWMYYOUOIBIX4',
   'snd': 'HNDUHBASQYQTWKHGQGH7UQCYKA2Y75HKTKK2TL4DBDWO3YAKQMV2Y4BVFE',
   'type': 'appl'}}}

### Asset Balances

In [11]:
print("Asset holding for user address: ")
print_asset_holding(algod_client, test_address, asset_id)


print("Asset holding for smart contract address: ")
print_asset_holding(algod_client, application_address, asset_id)

Asset holding for user address: 
Asset ID: 81470512
{
    "amount": 999999993,
    "asset-id": 81470512,
    "is-frozen": false
}
Asset holding for smart contract address: 
Asset ID: 81470512
{
    "amount": 7,
    "asset-id": 81470512,
    "is-frozen": false
}


### Algo Balances

In [12]:
account_info = algod_client.account_info(test_address)
print("Account balance test user: {} microAlgos".format(account_info.get('amount')) + "\n")


account_info = algod_client.account_info(application_address)
print("Account balance application address: {} microAlgos".format(account_info.get('amount')) + "\n")

Account balance test user: 12936993 microAlgos

Account balance application address: 20000003 microAlgos



### Testing the user still has custody over funds

In [13]:
def transfer_asset(private_key, address):
    #creator_private_key = mnemonic.to_private_key(creator_mnemonic)
    params = algod_client.suggested_params()

    # Use the AssetTransferTxn class to transfer assets and opt-in
    txn = transaction.AssetTransferTxn(
        sender=address,
        sp=params,
        receiver=application_address,
        amt=1,
        index=asset_id)
        
    stxn = txn.sign(private_key)
       
    txid = algod_client.send_transaction(stxn)
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid) 
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))    

    print_asset_holding(algod_client, test_address, asset_id)

def payment_txn(private_key, address):

    # build transaction
    params = algod_client.suggested_params()

    receiver = "HZ57J3K46JIJXILONBBZOHX6BKPXEM2VVXNRFSUED6DKFD5ZD24PMJ3MVA"
    amount = 1

    unsigned_txn = transaction.PaymentTxn(address, params, receiver, amount, None)

    # sign transaction
    signed_txn = unsigned_txn.sign(private_key)

    # submit transaction
    txid = algod_client.send_transaction(signed_txn)

    # wait for confirmation 
   
    result = wait_for_confirmation(algod_client, txid)  
   

    print("Transaction information: {}".format(json.dumps(result, indent=4)))


transfer_asset(test_private_key, test_address)
payment_txn(test_private_key, test_address)    

Waiting for confirmation...
Transaction 4CN4Q2CI2CI5HTC652S2L56IK6C7PX5NYMCFWURRPOCYGIYK6XAA confirmed in round 20707589.
Result confirmed in round: 20707589
Asset ID: 81470512
{
    "amount": 999999992,
    "asset-id": 81470512,
    "is-frozen": false
}
Waiting for confirmation...
Transaction PJTGXQSNYS3ZIHMQKTGVULWA752UREH4BCFPTURODRWWEG5SIQ5Q confirmed in round 20707591.
Transaction information: {
    "confirmed-round": 20707591,
    "pool-error": "",
    "txn": {
        "sig": "wL14qwImYTQL56nUpaWbZiE9ezSuLM+0ZzZH2d1Kfx3JWIIljKGgCHCN/1Wzhzj+9UdTpHhLbRoRbY3ItDLTBw==",
        "txn": {
            "amt": 1,
            "fee": 1000,
            "fv": 20707589,
            "gen": "testnet-v1.0",
            "gh": "SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=",
            "lv": 20708589,
            "rcv": "HZ57J3K46JIJXILONBBZOHX6BKPXEM2VVXNRFSUED6DKFD5ZD24PMJ3MVA",
            "snd": "HNDUHBASQYQTWKHGQGH7UQCYKA2Y75HKTKK2TL4DBDWO3YAKQMV2Y4BVFE",
            "type": "pay"
        }
  